# Install dependencies

In [57]:
!pip install pandas ipython-sql


In [58]:
!pip install beautifulsoup4 requests lxml

# Add imports

In [8]:
import pandas as pd

In [9]:
from bs4 import BeautifulSoup
import requests

# Create Datasets for all pharmacies in London separated by borough

## Download pharmacy list from https://opendata.nhsbsa.net/dataset/consolidated-pharmaceutical-list

In [13]:
# Consolidated Pharmaceutical List: 2023-24 Q4
# https://opendata.nhsbsa.net/dataset/240d142d-df82-4e97-b051-12371519e4e1/resource/d36c355a-631d-4ddb-bc34-5a0315565e12/download/consol_pharmacy_list_202324q4.csv

CONSOLIDATED_PHARMACEUTICAL_LIST_CSV = "https://opendata.nhsbsa.net/dataset/240d142d-df82-4e97-b051-12371519e4e1/resource/d36c355a-631d-4ddb-bc34-5a0315565e12/download/consol_pharmacy_list_202324q4.csv"
pharmacy_df = pd.read_csv(CONSOLIDATED_PHARMACEUTICAL_LIST_CSV)

## Generate London Pharmacies Dataframe

In [17]:
lon_health_wellbeing_boards = ["CITY OF LONDON", "KENSINGTON AND CHELSEA", "BRENT", "HAMMERSMITH AND FULHAM", "HOUNSLOW", "ISLINGTON", "SUTTON", "WALTHAM FOREST", "KINGSTON", "BARNET", "SOUTHWARK", "CAMDEN", "HILLINGDON", "WESTMINSTER", "CROYDON", "HARINGEY", "GREENWICH", "LEWISHAM", "BARKING AND DAGENHAM", "MERTON", "LAMBETH", "EALING", "BEXLEY", "HARROW", "REDBRIDGE", "HAVERING", "HACKNEY", "TOWER HAMLETS"]

In [128]:
lon_pharmacy_df = pharmacy_df[["PHARMACY_ODS_CODE_(F-CODE)", "HEALTH_AND_WELLBEING_BOARD", "PHARMACY_TRADING_NAME", "ADDRESS_FIELD1", "ADDRESS_FIELD2", "ADDRESS_FIELD3", "ADDRESS_FIELD4", "POST_CODE"]]

In [130]:
lon_pharmacy_df = lon_pharmacy_df.loc[pharmacy_df.HEALTH_AND_WELLBEING_BOARD.isin(lon_health_wellbeing_boards)]

## Create Dataframe for each London borough from London Pharmacies Dataframe

In [132]:
def create_borough_df(borough_name):
    return lon_pharmacy_df.loc[lon_pharmacy_df.HEALTH_AND_WELLBEING_BOARD.isin([borough_name])]

In [133]:
# Create df for each London health and wellbeing board

barnet_pharmacy_df = create_borough_df("BARNET")
barking_and_dagenham_pharmacy_df = create_borough_df("BARKING AND DAGENHAM")
bexley_pharmacy_df = create_borough_df("BEXLEY")
brent_pharmacy_df = create_borough_df("BRENT")

camden_pharmacy_df = create_borough_df("CAMDEN")
city_of_london_pharmacy_df = create_borough_df("CITY OF LONDON")
croydon_pharmacy_df = create_borough_df("CROYDON")

ealing_pharmacy_df = create_borough_df("EALING")

greenwich_pharmacy_df = create_borough_df("GREENWICH")

hackney_pharmacy_df = create_borough_df("HACKNEY")
haringey_pharmacy_df = create_borough_df("HARINGEY")
hammersmith_and_fulham_pharmacy_df = create_borough_df("HAMMERSMITH AND FULHAM")
harrow_pharmacy_df = create_borough_df("HARROW")
havering_pharmacy_df = create_borough_df("HAVERING")
hillingdon_pharmacy_df = create_borough_df("HILLINGDON")
hounslow_pharmacy_df = create_borough_df("HOUNSLOW")

islington_pharmacy_df = create_borough_df("ISLINGTON")

kensington_and_chelsea_pharmacy_df = create_borough_df("KENSINGTON AND CHELSEA")
kingston_pharmacy_df = create_borough_df("KINGSTON")

lambeth_pharmacy_df = create_borough_df("LAMBETH")
lewisham_pharmacy_df = create_borough_df("LEWISHAM")

merton_pharmacy_df = create_borough_df("MERTON")

redbridge_pharmacy_df = create_borough_df("REDBRIDGE")

southwark_pharmacy_df = create_borough_df("SOUTHWARK")
sutton_pharmacy_df = create_borough_df("SUTTON")

tower_hamlets_pharmacy_df = create_borough_df("TOWER HAMLETS")

waltham_forest_pharmacy_df = create_borough_df("WALTHAM FOREST")
westminster_pharmacy_df = create_borough_df("WESTMINSTER")

# Scrape phone numbers from NHS and add to data set

In [248]:
def get_phone_number(pharmacy_ods_code, pharmacy_trading_name):
    
    pharmacy_trading_name = pharmacy_trading_name.lower()
    pharmacy_trading_name = pharmacy_trading_name.replace(" ", "-")
    
    query = f"https://www.nhs.uk/services/pharmacy/{pharmacy_trading_name}/{pharmacy_ods_code}"
    response = requests.get(query)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    return soup.find("p", {"id": "contact_info_panel_phone_text"}).getText()

def phone_number_with_country_code_and_no_spaces(phone_number):
    phone_number = phone_number.replace(" ", "")
    return phone_number.replace("0", "+44", 1)

In [246]:
ods_code, pharmacy_name = tower_hamlets_pharmacy_df[["PHARMACY_ODS_CODE_(F-CODE)", "PHARMACY_TRADING_NAME"]].iloc[3]

020 7247 6729


In [231]:
for row in tower_hamlets_pharmacy_df.iterrows():
    
    phone_number = get_phone_number(ods_code, pharmacy_name)
    phone_number = phone_number_with_country_code_and_no_spaces(phone_number)
    print(phone_number)

TypeError: tuple indices must be integers or slices, not list

In [249]:
# transform phone number for df
phone_number = get_phone_number(ods_code, pharmacy_name)
print(phone_number)
phone_number = phone_number_with_country_code_and_no_spaces(phone_number)
print(phone_number)

020 7247 6729
+442072476729


In [218]:
row_index = tower_hamlets_pharmacy_df.index[3]
tower_hamlets_pharmacy_df.loc[row_index, 'PHONE_NUMBER'] = phone_number

# Create CSV for each borough

In [219]:
import inspect

def retrieve_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    names = [var_name for var_name, var_val in callers_local_vars if var_val is var]
    if len(names) > 0:
            return names[0]

In [220]:
borough_dataframes = [
    barnet_pharmacy_df,
    barking_and_dagenham_pharmacy_df,
    bexley_pharmacy_df,
    brent_pharmacy_df,
    camden_pharmacy_df,
    city_of_london_pharmacy_df,
    croydon_pharmacy_df,
    ealing_pharmacy_df,
    greenwich_pharmacy_df,
    hackney_pharmacy_df,
    haringey_pharmacy_df,
    hammersmith_and_fulham_pharmacy_df,
    harrow_pharmacy_df,
    havering_pharmacy_df,
    hillingdon_pharmacy_df,
    hounslow_pharmacy_df,
    islington_pharmacy_df,
    kensington_and_chelsea_pharmacy_df,
    kingston_pharmacy_df,
    lambeth_pharmacy_df,
    lewisham_pharmacy_df,
    merton_pharmacy_df,
    redbridge_pharmacy_df,
    southwark_pharmacy_df,
    sutton_pharmacy_df,
    tower_hamlets_pharmacy_df,
    waltham_forest_pharmacy_df,
    westminster_pharmacy_df
]

In [221]:
for name in borough_dataframes:
    df_var_name = retrieve_name(name)
    csv_name = df_var_name.replace('_pharmacy_df', '')
    name.to_csv('./' + csv_name + '_pharmacies.csv')